# Eigen y Python

En linux se puede instalar Eigen con el paquete libeigen3-dev. También se puede bajar la carpeta eigen para poder elegir bien la versión.

In [2]:
# !git clone https://gitlab.com/libeigen/eigen.git
!apt-get install libeigen3-dev

E: Could not open lock file /var/lib/dpkg/lock-frontend - open (13: Permission denied)
E: Unable to acquire the dpkg frontend lock (/var/lib/dpkg/lock-frontend), are you root?


# Ejemplo Eigen básico

%%file file_name, copia todo el contendio de la celda y lo agrega al archivo file_name

In [4]:
%%file eigen_test.cpp


#include <iostream>
#include <eigen3/Eigen/Dense>

int main()
{
  Eigen::Matrix2d a;
  a << 1, 2,
       3, 4;
  Eigen::MatrixXd b(2,2);   // matriz de tamanio dinamico
  b << 2, 3,
       1, 4;
  std::cout << "a + b =\n" << a + b << std::endl;
  std::cout << "a - b =\n" << a - b << std::endl;
  std::cout << "Doing a += b;" << std::endl;
  a += b;
  std::cout << "Now a =\n" << a << std::endl;
  Eigen::Vector3d v(1,2,3);
  Eigen::Vector3d w(1,0,0);
  std::cout << "-v + w - v =\n" << -v + w - v << std::endl;
}

Overwriting eigen_test.cpp


In [5]:
%%bash

# g++ -I eigen eigen_test.cpp -o out
g++ eigen_test.cpp -o out
./out

a + b =
3 5
4 8
a - b =
-1 -1
 2  0
Doing a += b;
Now a =
3 5
4 8
-v + w - v =
-1
-4
-6


## Ejemplo con una función de producto matriz vector

In [6]:
%%file eigen_types_test.cpp

#include <iostream>
#include <eigen3/Eigen/Dense>

using Eigen::MatrixXd;
using Eigen::VectorXd;

VectorXd matrix_vector_multiplication(const MatrixXd& matrix, const VectorXd& vector) {
    return matrix * vector;
}


int main() {
    // Example usage
    MatrixXd A(3, 3);
    A << 1, 2, 3,
         4, 5, 6,
         7, 8, 9;

    VectorXd b(3);
    b << 1, 2, 3;

    VectorXd c = matrix_vector_multiplication(A, b);

    std::cout << "Result of matrix-vector multiplication: " << c.transpose() << std::endl;

    return 0;
}

Writing eigen_types_test.cpp


## Compilamos

In [7]:
%%bash
# g++ -Ieigen eigen_types_test.cpp -o out
g++ eigen_types_test.cpp -o out
./out

Result of matrix-vector multiplication: 14 32 50


## Comparamos con numpy

In [8]:
import numpy as np
A = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]], dtype=np.float64)
b = np.array([1, 2, 3], dtype=np.float64)
A@b

array([14., 32., 50.])

# Opción con entrada y salida de archivos de texto

In [9]:
%%file eigen_types_iofile_test.cpp

#include <iostream>
#include <fstream>
#include <eigen3/Eigen/Dense>

using Eigen::MatrixXd;
using Eigen::VectorXd;

VectorXd matrix_vector_multiplication(const MatrixXd& matrix, const VectorXd& vector) {
    return matrix * vector;
}

int main(int argc, char** argv) {
    if (argc != 3) {
        std::cerr << "Usage: " << argv[0] << " input_file output_file" << std::endl;
        return 1;  // codigo 1 para cantidad incorrecta de argumentos.
    }

    const char* input_file = argv[1];
    const char* output_file = argv[2];

    std::ifstream fin(input_file);
    if (!fin.is_open()) {
        std::cerr << "Error: could not open input file " << input_file << std::endl;
        return 1;
    }

    // Read matrix and vector from file
    int nrows, ncols;
    fin >> nrows >> ncols;

    MatrixXd A(nrows, ncols);
    for (int i = 0; i < nrows; i++) {
        for (int j = 0; j < ncols; j++) {
            fin >> A(i, j);
        }
    }

    VectorXd b(ncols);
    for (int i = 0; i < ncols; i++) {
        fin >> b(i);
    }

    fin.close();

    // Perform matrix-vector multiplication
    VectorXd c = matrix_vector_multiplication(A, b);

    // Write result to output file
    std::ofstream fout(output_file);
    if (!fout.is_open()) {
        std::cerr << "Error: could not open output file " << output_file << std::endl;
        return 1;
    }

    fout << c.transpose() << std::endl;

    fout.close();

    return 0;
}


Writing eigen_types_iofile_test.cpp


## Escribimos un archivo de texto con numpy

!rm input_data.txt  # borra el archivo si es que ya existe.

with open('input_data.txt','a') as f: line opens the file input_data.txt in append mode ('a').

In [10]:
A = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]], dtype=np.float64)
b = np.array([1, 2, 3], dtype=np.float64)

# borramos el archivo input si ya existe
!rm input_data.txt 

with open('input_data.txt','a') as f: 
    f.write(f"{A.shape[0]} {A.shape[1]}\n")
    np.savetxt(f,A, newline="\n")
    # np.savetxt(f,b.reshape(1,-1), fmt='%1.3f', newline="\n")
    np.savetxt(f,b, fmt='%1.3f', newline="\n")
!cat input_data.txt

rm: cannot remove 'input_data.txt': No such file or directory
3 3
1.000000000000000000e+00 2.000000000000000000e+00 3.000000000000000000e+00
4.000000000000000000e+00 5.000000000000000000e+00 6.000000000000000000e+00
7.000000000000000000e+00 8.000000000000000000e+00 9.000000000000000000e+00
1.000 2.000 3.000


## Compilamos y ejecutamos

In [ ]:
%%bash

g++ eigen_types_iofile_test.cpp -o out
./out input_data.txt output_data.txt
cat output_data.txt

14 32 50


In [ ]:
np.loadtxt('output_data.txt')

array([14., 32., 50.])

# Usando Python ctypes

In [ ]:
%%file eigen_ctypes_test.cpp

#include <iostream>
#include <eigen3/Eigen/Dense>
#include <dlfcn.h>

using namespace Eigen;

extern "C" {
    void matrix_vector_multiply(double* matrix, double* vector, double* result, int rows, int cols) {
        Map<MatrixXd> mat(matrix, rows, cols);
        Map<VectorXd> vec(vector, cols);
        Map<VectorXd> res(result, rows);
        res = mat * vec ;
    }
}

Writing eigen_ctypes_test.cpp


In [ ]:
!rm eigen_ctypes_test.so # borramos por las dudas
!g++ -shared -fPIC -Llibdl -o eigen_ctypes_test.so eigen_ctypes_test.cpp
!ls # chequeamos que este la lib

rm: cannot remove 'eigen_ctypes_test.so': No such file or directory
eigen_ctypes_test.cpp  eigen_types_iofile_test.cpp  out
eigen_ctypes_test.so   eigen_types_test.cpp	    output_data.txt
eigen_test.cpp	       input_data.txt		    sample_data


In [ ]:
import ctypes
import numpy as np

class sharedlib():
    dlclose = ctypes.CDLL(None).dlclose  # This WON'T work on Win
    dlclose.argtypes = (ctypes.c_void_p,)

    def __init__(self, path, method, *args):
        self.lib = ctypes.cdll.LoadLibrary(f'./{path}')

        # Se explicitan los tipos de los argumentos para el método deseado
        self.method_object = getattr(self.lib, method)
        self.method_object.argtypes = args

    def __call__(self, *args):
        return self.method_object(*args)

    def unload(self):
        while self.dlclose(self.lib._handle)!=-1:
            pass

lib = sharedlib('eigen_ctypes_test.so', 'matrix_vector_multiply',
                                                                ctypes.POINTER(ctypes.c_double),
                                                                ctypes.POINTER(ctypes.c_double),
                                                                ctypes.POINTER(ctypes.c_double),
                                                                ctypes.c_int,
                                                                ctypes.c_int
                                                            )

# Creamos la matriz de entrada
A = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]], dtype=np.float64)
# Eigen la mapea transpuesta a la información del puntero así que hay que pasarlo a orden tipo Fortran
A = np.asfortranarray(A)
b = np.array([1, 2, 3], dtype=np.float64)

# Definimos el vector donde se van a guardar los resultados
result = np.zeros((3,), dtype=np.float64)

# Llamamos a nuestra función en C++ pasando los argumentos de entrada
lib(
    A.ctypes.data_as(ctypes.POINTER(ctypes.c_double)),
    b.ctypes.data_as(ctypes.POINTER(ctypes.c_double)),
    result.ctypes.data_as(ctypes.POINTER(ctypes.c_double)),
    ctypes.c_int(A.shape[0]),
    ctypes.c_int(A.shape[1])
)

lib.unload() # para poder recompilar la lib hay que cerrarla

result

array([14., 32., 50.])

# Cython

In [ ]:
%reload_ext cython

In [ ]:
%%cython -a --verbose

import math
# use C math functions
from libc.math cimport sin, cos

# use C types instead of Python types
def r_cython(double[:] x_vec, double[:] y_vec):
    cdef double s = 0
    cdef int i
    for i in range(len(x_vec)):
        s += cos(x_vec[i])*sin(y_vec[i])
    return s

In [ ]:
x = np.ones(10)
r_cython(x, x)

4.546487134128409

In [ ]:
!python -m pip install eigency
%reload_ext cython

In [ ]:
%%writefile functions.h
Eigen::MatrixXd function_w_mat_arg(const Eigen::Map<Eigen::MatrixXd> &mat) {
    std::cout << mat << "\n";
    return mat;
}

Overwriting functions.h


In [ ]:
%%cython -a --verbose -+ -I /usr/local/lib/python3.10/dist-packages/numpy/core/include/ -I /usr/include/eigen3 -I ./

import math
# use C math functions
from libc.math cimport sin, cos
from eigency.core cimport *

cdef extern from "functions.h":
     cdef MatrixXd _function_w_mat_arg "function_w_mat_arg"(Map[MatrixXd] &)

# use C types instead of Python types
def function_w_mat_arg(np.ndarray array):
    return ndarray(_function_w_mat_arg(Map[MatrixXd](array)))

[1/1] Cythonizing /root/.cache/ipython/cython/_cython_magic_583902f51c5075cee7617670ace3d5165134481a.pyx


INFO:root:building '_cython_magic_583902f51c5075cee7617670ace3d5165134481a' extension
INFO:root:x86_64-linux-gnu-gcc -Wno-unused-result -Wsign-compare -DNDEBUG -g -fwrapv -O2 -Wall -g -fstack-protector-strong -Wformat -Werror=format-security -g -fwrapv -O2 -fPIC -I/usr/local/lib/python3.10/dist-packages/eigency -I/usr/local/lib/python3.10/dist-packages/numpy/core/include/ -I/usr/include/eigen3 -I./ -I/usr/include/python3.10 -c /root/.cache/ipython/cython/_cython_magic_583902f51c5075cee7617670ace3d5165134481a.cpp -o /root/.cache/ipython/cython/root/.cache/ipython/cython/_cython_magic_583902f51c5075cee7617670ace3d5165134481a.o
INFO:root:x86_64-linux-gnu-g++ -shared -Wl,-O1 -Wl,-Bsymbolic-functions -Wl,-Bsymbolic-functions -g -fwrapv -O2 /root/.cache/ipython/cython/root/.cache/ipython/cython/_cython_magic_583902f51c5075cee7617670ace3d5165134481a.o -L/usr/lib/x86_64-linux-gnu -o /root/.cache/ipython/cython/_cython_magic_583902f51c5075cee7617670ace3d5165134481a.cpython-310-x86_64-linux-gnu.

Content of stderr:
In file included from /usr/local/lib/python3.10/dist-packages/numpy/core/include/numpy/ndarraytypes.h:1929,
                 from /usr/local/lib/python3.10/dist-packages/numpy/core/include/numpy/ndarrayobject.h:12,
                 from /usr/local/lib/python3.10/dist-packages/numpy/core/include/numpy/arrayobject.h:5,
                 from /root/.cache/ipython/cython/_cython_magic_583902f51c5075cee7617670ace3d5165134481a.cpp:1268:
/usr/local/lib/python3.10/dist-packages/numpy/core/include/numpy/npy_1_7_deprecated_api.h:17:2: warning: #warning "Using deprecated NumPy API, disable it with " "#define NPY_NO_DEPRECATED_API NPY_1_7_API_VERSION" [-Wcpp]
   17 | #warning "Using deprecated NumPy API, disable it with " \
      |  ^~~~~~~

In [ ]:
function_w_mat_arg(np.ones((3,3)))

array([[1., 1., 1.],
       [1., 1., 1.],
       [1., 1., 1.]])

# pybind11

In [ ]:
if "google.colab" in str(get_ipython()):
    !pip install git+https://github.com/aldanor/ipybind.git -qqq
%load_ext ipybind

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.0/235.0 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 39.7 MB/s eta 0:00:00


In [ ]:
%%pybind11

#include <pybind11/numpy.h>
#include <math.h>
#include <eigen3/Eigen/Dense>
#include <iostream>
PYBIND11_PLUGIN(example) {
    py::module m("example");
    m.def("r_pybind", [](const py::array_t<double>& x, const py::array_t<double>& y) {
        double sum{0};
        auto rx{x.unchecked<1>()};
        auto ry{y.unchecked<1>()};
        for (py::ssize_t i = 0; i < rx.shape(0); i++){
            sum += std::cos(rx[i])*std::sin(ry[i]);
        }
        Eigen::Matrix2d a;
        a << 1, 2,
            3, 4;

        return sum;

    });
    return m.ptr();
}

In [ ]:
r_pybind(x, x)